In [1]:
import pandas as pd
import re
pd.set_option('display.max_rows' , 200)

In [2]:
company_name = 'tcs'.upper()

In [3]:
share_price_in = 1000000;

In [4]:
def calculate_growth(initial , final) :
    if not initial :
        return 1
    return round(((final - initial) / initial) * 100,2)

In [5]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [6]:
def vertical_analysis(item , divider) :
    return (item/divider)
    

In [7]:
def round_off_to(num , limit) :
    return round(num,limit)

In [8]:
def count_leading_space(s): 
    match = re.search(r"^\s*", s) 
    return 0 if not match else match.end()


In [9]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [10]:
styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", 'white'),
                              ("background", '#308D46'),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [11]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [12]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [13]:
income_statement.rename(columns={company_name+'_income-statement_Annual_As_Originally_Reported' : 'items'} , inplace=True)
balance_sheet.rename(columns={company_name+'_balance-sheet_Annual_As_Originally_Reported' : 'items'} , inplace=True)
cashflow_statement.rename(columns={company_name+'_cash-flow_Annual_As_Originally_Reported' : 'items'} , inplace=True)

In [14]:
basic_eps_filter = income_statement['items'].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [15]:
WASO_filter = income_statement['items'].eq('Basic Weighted Average Shares Outstanding') | income_statement['items'].eq('Diluted Weighted Average Shares Outstanding') | income_statement['items'].eq('Basic WASO') | income_statement['items'].eq('Diluted WASO')
waso_pos = income_statement[WASO_filter].index

In [16]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [17]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [18]:
temp_df = temp_df.apply(get_share_price_in)

In [19]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [20]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [21]:
income_statement.loc[1,'items'] = income_statement.loc[1,'items'].strip()
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue 

In [22]:
is_highlighted_indexes = income_statement.loc[0:basic_eps_pos - 1,'items'].copy()
is_highlighted_indexes = is_highlighted_indexes.apply(lambda x: x[0] != ' ')
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [23]:
income_statement['items'] = income_statement['items'].apply(lambda x: x.strip())

In [24]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [25]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [26]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is['items'] = income_statement.loc[0:basic_eps_pos - 1,'items'].copy()


In [27]:
total_revenue_filter = income_statement['items'].eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [28]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
    

In [29]:
#def row_style(row):
#    if row['items'] in vertical_analysis_is.loc[is_highlighted_indexes,'items'].values:
#        return pd.Series('background-color: hsl(47,83%,91%);', row.index)
#    else :
#        return pd.Series('', row.index)
#vertical_analysis_is.style.apply(row_style,axis=1)

In [30]:
vertical_analysis_is.loc[is_highlighted_indexes].style.set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

,items,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Total Revenue,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
1,Gross Profit,57.62%,59.73%,55.06%,57.31%,45.38%,43.87%,45.03%,44.02%,47.08%,47.71%,46.86%
7,Operating Income/Expenses,-30.62%,-30.63%,-31.08%,-30.93%,-19.57%,-18.93%,-19.46%,-19.44%,-21.09%,-22.37%,-22.70%
26,Total Operating Profit/Loss,27.00%,29.09%,23.98%,26.37%,25.81%,24.94%,25.57%,24.58%,26.00%,25.34%,24.16%
27,"Non-Operating Income/Expense, Total",1.72%,1.96%,3.80%,2.78%,3.45%,2.75%,2.81%,2.34%,0.66%,1.62%,1.15%
45,Pretax Income,28.72%,31.05%,27.79%,29.16%,29.26%,27.69%,28.38%,26.92%,26.65%,26.95%,25.30%
46,Provision for Income Tax,-6.37%,-7.42%,-6.59%,-6.72%,-6.91%,-6.67%,-6.83%,-6.24%,-6.82%,-6.90%,-6.48%
49,Net Income before Extraordinary Items and Discontinued Operations,22.35%,23.63%,21.19%,22.44%,22.34%,21.02%,21.55%,20.67%,19.83%,20.05%,18.82%
50,Net Income after Extraordinary Items and Discontinued Operations,22.35%,23.63%,21.19%,22.44%,22.34%,21.02%,21.55%,20.67%,19.83%,20.05%,18.82%
51,Non-Controlling/Minority Interests,-0.25%,-0.21%,-0.22%,-0.08%,-0.06%,-0.04%,-0.06%,-0.07%,-0.08%,-0.06%,-0.07%
